In [1]:
import os
import torch
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import psutil
import h5py
import os
import numpy as np
import time
import copy
from torchsummary import summary
from torch import optim
from torch.optim import lr_scheduler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

!pip install vit-pytorch
from vit_pytorch import ViT


try:
  import google.colab
  COLAB_ENV = True
except:
  COLAB_ENV = False

In [2]:
if COLAB_ENV:
  start = "/" + os.path.join("content","drive","MyDrive","datasets")
  from google.colab import drive
  drive.mount('/content/drive')
else:
  start = "data"

In [3]:
data_filepath = os.path.join(start, "CERN", "ParticleClassification")
photon_file = "SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"
electron_file = "SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"

photon_filepath = os.path.join(data_filepath, photon_file)
electron_filepath = os.path.join(data_filepath, electron_file)

electronFile = h5py.File(electron_filepath, "r")
photonFile = h5py.File(photon_filepath, "r")

class PhotonElectronDataset(Dataset):
    def __init__(self, _electron_filepath, _photon_filepath, transform=None, target_transform=None, low = 0, high = 1):
        global seed
        np.random.seed(seed)

        electronFile = h5py.File(_electron_filepath, "r")
        photonFile = h5py.File(_photon_filepath, "r")

        ElectronX, ElectronY = electronFile["X"], electronFile["y"]
        PhotonX, PhotonY = photonFile["X"], photonFile["y"]

        indexes = np.random.permutation(ElectronX.shape[0])

        _low = int(low * len(indexes))
        _high = int(high * len(indexes))

        self.X = np.concatenate([ElectronX[_low:_high, :,:,:], PhotonX[_low:_high, :,:,:]])
        self.Y = np.concatenate([ElectronY[_low:_high], PhotonY[_low:_high]])
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        #image = torch.from_numpy(self.X[idx, :, :, :])
        image = self.X[idx, :, :, :]

        label = self.Y[idx,]
        if self.transform:
            image = self.transform(image)
        return image, label


In [4]:
seed = 1231
train_test_split = 0.8
bs = 16

print(f"Before {psutil.virtual_memory()[1] / (1e9)}")
training_data = PhotonElectronDataset(electron_filepath, photon_filepath, transform=transforms.Compose([transforms.ToTensor()]), low=0, high=train_test_split)
validation_data = PhotonElectronDataset(electron_filepath, photon_filepath, transform=transforms.Compose([transforms.ToTensor()]), low=train_test_split, high=1)

print(f"Training dataset : {len(training_data)}")
print(f"Validation dataset : {len(validation_data)}")

train_dataloader = DataLoader(training_data, batch_size=bs, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=bs, shuffle=True)

print(f"After {psutil.virtual_memory()[1] / (1e9)}")

Before 11.40256768
Training dataset : 398400
Validation dataset : 99600
After 7.312928768


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Device Used : {device}")

def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, saveTo, num_epochs=25):
    since = time.time()

    returnDict = {"trainingLoss" : [], "validationLoss" : [], "trainingAccuracy" : [], "validationAccuracy" : [],
                  "epochs" : [], "trainingAUC" : [], "validationAUC" : []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        returnDict["epochs"].append(epoch)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            since2 = time.time()
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            y = torch.Tensor()
            pred_y = torch.Tensor()

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.long()

                y = torch.cat([y, labels.cpu()])
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    probs, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    pred_y = torch.cat([pred_y, outputs[:, 1].cpu()])

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            AUC_ROC = roc_auc_score(y.detach().numpy(), pred_y.detach().numpy())

            if phase == "train":
                returnDict["trainingLoss"].append(epoch_loss)
                returnDict["trainingAccuracy"].append(epoch_acc.cpu().item())
                returnDict["trainingAUC"].append(AUC_ROC)
            else:
                returnDict["validationLoss"].append(epoch_loss)
                returnDict["validationAccuracy"].append(epoch_acc.cpu().item())
                returnDict["validationAUC"].append(AUC_ROC)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} AUC ROC : {AUC_ROC:.4f} Time taken : {round(time.time() - since2, 2)}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model, saveTo)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    returnDict["best_model"] = model
    return returnDict



Device Used : cuda:0


In [6]:
def plotResults(results):
    plt.figure(figsize=(16,12))
    plt.subplot(3,1, 1)
    plt.plot(results['epochs'], results['trainingLoss'], label="training Loss")
    plt.plot(results['epochs'], results['validationLoss'], label="validation Loss")
    plt.legend()

    plt.subplot(3,1, 2)
    plt.plot(results['epochs'], results['trainingAccuracy'], label="training Accuracy")
    plt.plot(results['epochs'], results['validationAccuracy'], label="validation Accuracy")
    plt.legend()


    plt.subplot(3,1, 3)
    plt.plot(results['epochs'], results['trainingAUC'], label="training ROC")
    plt.plot(results['epochs'], results['validationAUC'], label="validation AUC ROC")
    plt.legend()

    plt.show()

In [7]:
class BasicCNNBlock(nn.Module):
    def __init__(self):
        super(BasicCNNBlock, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Dropout(0.25),

            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),

            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Flatten(),
            nn.Linear(1024, 64),
        )

    def forward(self, x):
        logits = self.model(x)
        return logits


class splittedCNN(nn.Module):
    def __init__(self):
        super(splittedCNN, self).__init__()

        self.cnn1 = BasicCNNBlock()
        self.cnn2 = BasicCNNBlock()

        self.pred = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,2),
            nn.Softmax(1)
        )

    def forward(self, x):

        y1 = self.cnn1(x[:, 0, :, :].reshape(-1, 1, 32, 32))
        y2 = self.cnn2(x[:, 1, :, :].reshape(-1, 1, 32, 32))
        y3 = torch.cat([y1, y2], axis=1)
        return self.pred(y3)


model = splittedCNN()
model = model.to(device)

print(summary(model, input_size=(2, 32, 32)))

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

results = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,{"train" : train_dataloader, "val": validation_dataloader},
                       {"train" : len(training_data), "val" : len(validation_data)}, saveTo="basicNN", num_epochs=50)

plotResults(results)

/home/ache/anaconda3/lib/python3.9/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             640
              ReLU-2           [-1, 64, 32, 32]               0
         MaxPool2d-3           [-1, 64, 16, 16]               0
           Dropout-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,928
              ReLU-6           [-1, 64, 16, 16]               0
         MaxPool2d-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,928
              ReLU-9             [-1, 64, 8, 8]               0
        MaxPool2d-10             [-1, 64, 4, 4]               0
          Flatten-11                 [-1, 1024]               0
           Linear-12                   [-1, 64]          65,600
    BasicCNNBlock-13                   [-1, 64]               0
           Conv2d-14           [-1, 64,

/home/ache/anaconda3/lib/python3.9/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


val Loss: 0.5966 Acc: 0.6967 AUC ROC : 0.7508 Time taken : 16.17

Epoch 1/49
----------


/home/ache/anaconda3/lib/python3.9/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


KeyboardInterrupt: 